# Java Threads and Concurrency Utilities

## Chapter - 4
## Additional Thread Capabilities

#### Thread Groups
The JDK documentation for ThreadGroup states that a thread group “represents a set of threads. In addition, a thread group can also include other thread groups. The thread groups form a tree in which every thread group except the initial thread group has a parent.”

Although ThreadGroup appears to be a very useful, you should largely avoid this class for the following reasons:
* The most useful ThreadGroup methods are void suspend(), void resume(), and void stop(). These methods have been deprecated.
* ThreadGroup isn’t thread-safe.

However, you should still know about ThreadGroup because of its contribution in handling exceptions that are thrown while a thread is executing.

In [1]:
Runnable r = () -> {
            int x = 1 / 0; // Line 10
        };

In [2]:
Thread thd = new Thread(r);
thd.start();

Exception in thread "Thread-1" java.lang.ArithmeticException: / by zero
	at REPL.$JShell$12.lambda$do_it$$0($JShell$12.java:18)
	at java.base/java.lang.Thread.run(Thread.java:832)


In [3]:
Runnable rWithExceptionHandling = () -> {
            int x = 1 / 0; // Line 10
        };

In [4]:
Thread thdWithExceptionHandling = new Thread(rWithExceptionHandling);
Thread.UncaughtExceptionHandler uceh;
uceh = new Thread.UncaughtExceptionHandler() {
    @Override
    public void uncaughtException(Thread t, Throwable e) {
        System.out.println("Caught throwable " + e + " for thread " + t);
    }
};

thdWithExceptionHandling.setUncaughtExceptionHandler(uceh);

In [5]:
uceh = new Thread.UncaughtExceptionHandler() {
    @Override
    public void uncaughtException(Thread t, Throwable e) {
        System.out.println("Default uncaught exception handler");
        System.out.println("Caught throwable " + e + " for thread " + t);
    }
};
thdWithExceptionHandling.setDefaultUncaughtExceptionHandler(uceh);

In [6]:
thdWithExceptionHandling.start();

Caught throwable java.lang.ArithmeticException: / by zero for thread Thread[Thread-2,5,main]


#### When an exception is thrown out of the run() method, the thread terminates and the following activities take place:
* The Java virtual machine (JVM) looks for an instance of Thread. UncaughtExceptionHandler installed via Thread’s void setUncau ghtExceptionHandler(Thread.UncaughtExceptionHandler eh) method. When this handler is found, it passes execution to the instance’svoid uncaughtException(Thread t, Throwable e) method, where t identifies the Thread object of the thread that threw the exception, and e identifies the thrown exception or error—perhaps a java.lang.OutOfMemoryError object was thrown. If uncaughtException() throws an exception/error, the exception/error is ignored by the JVM.
* Assuming that setUncaughtExceptionHandler() was not called to install a handler, the JVM passes control to the associated ThreadGroupobject’suncaughtException(Thread t, Throwable e) method. Assuming that ThreadGroup was not extended and that its uncaughtException() method was not overridden to handle the exception, uncaughtException() passes control to the parent ThreadGroup object’s uncaughtException() method when a parent ThreadGroup is present. Otherwise, it checks to see if a default uncaught exception handler has been installed (viaThread’sstatic void setDefaultUncaught ExceptionHandler(Thread.UncaughtExceptionHandler handler) method). If a default uncaught exception handler has been installed, its uncaughtException() method is called with the same two arguments. Otherwise, uncaughtException() checks its Throwable argument to determine if it’s an instance of java.lang.ThreadDeath. If so, nothing special is done. Otherwise, as Listing 4-1’s exception message shows, a message containing the thread’s name, as returned from the thread’s getName() method, and a stack backtrace, using the Throwable argument’s printStackTrace() method, is printed to the standard error stream.

<pre class="brush:java"> 
public class ThreadLocalDemo {
    private static volatile ThreadLocal<String> userID = new ThreadLocal<>();

    public static void main(String[] args) {
        Runnable r = () -> {
            String name = Thread.currentThread().getName();
            if (name.equals("A"))
                userID.set("foxtrot");
            else
                userID.set("charlie");
            System.out.println(name + " " + userID.get());
        };
        
        Thread thdA = new Thread(r);
        thdA.setName("A");
        Thread thdB = new Thread(r);
        thdB.setName("B");
        thdA.start();
        thdB.start();
    }
}
</pre>

In [7]:
static volatile ThreadLocal<String> userID = new ThreadLocal<>();

In [8]:
Runnable r = () -> {
            String name = Thread.currentThread().getName();
            if (name.equals("A"))
                userID.set("foxtrot");
            else
                userID.set("charlie");
            System.out.println(name + " " + userID.get());
        };

In [9]:
Thread thdA = new Thread(r);
        thdA.setName("A");
        Thread thdB = new Thread(r);
        thdB.setName("B");
        thdA.start();
        thdB.start();

A foxtrot
B charlie


<pre class="brush:java"> 
TimerTask task = new TimerTask() {
            @Override
            public void run() {
                System.out.println("alarm going off");
                System.exit(0);
            }
        };
</pre"> 

<pre class="brush:java"> 
Timer timer = new Timer();
timer.schedule(task, 10); // Execute one-shot timer task after
                          // 2-second delay.
</pre> 

<pre class="brush:java"> 
TimerTask task = new TimerTask()
                       {
                          @Override
                          public void run()
                          {
                             System.out.println(System.currentTimeMillis());
                          }
                       };
</pre>

<pre class="brush:java"> 
Timer timer = new Timer();
timer.schedule(task, 0, 1000);
</pre>

#### Timer declares the following constructors:
* __Timer():__ Create a new timer whose task-execution thread doesn’t
run as a daemon thread.
* __Timer(boolean isDaemon):__ Create a new timer whose task- execution thread may be specified to run as a daemon (pass true to isDaemon). A daemon thread is called for scenarios where the timer will be used to schedule repeating “maintenance activities,” which must be performed for as long as the application is running, but shouldn’t prolong the application’s lifetime.
* __Timer(String name):__ Create a new timer whose task-execution thread has the specified name. The task-execution thread doesn’t run as a daemon thread. This constructor throws java.lang. NullPointerException when name is null.
* __Timer(String name, boolean isDaemon):__ Create a new timer whose task-execution thread has the specified name and which may run as a daemon thread. This constructor throws NullPointerException when name is null.

#### Timer also declares the following methods:
* __void cancel():__ Terminate this timer, discarding any currently scheduled timer tasks. This method doesn’t interfere with a currently executing timer task (when it exists). After a timer has been terminated, its execution thread terminates gracefully and no more timer tasks may be scheduled on it. (Calling cancel() from within the run() method of a timer task that was invoked by this timer absolutely guarantees that the ongoing task execution is the last task execution that will ever be performed by this timer.) This method may be called repeatedly; the second and subsequent calls have no effect.
* __int purge():__ Remove all canceled timer tasks from this timer’s queue and return the number of timer tasks that have been removed. Calling purge() has no effect on the behavior of the timer, but eliminates references to the canceled timer tasks from the queue. When there are no external references to these timer tasks, they become eligible for garbage collection. (Most applications won’t need to call this method, which is designed for use by the rare application that cancels a large number of timer tasks. Calling purge() trades time for space: this method’s runtime may be proportional to n + c * log n, where n is the number of timer tasks in the queue and c is the number of canceled timer tasks.) It’s permissible to call purge() from within a timer task scheduled on this timer.
* __void schedule(TimerTask task, Date time):__ Schedule task for execution at time. When time is in the past, task is scheduled for immediate execution. This method throws java.lang. IllegalArgumentException when time.getTime() is negative; java.lang.IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task- execution thread terminated; and NullPointerException when task or time is null.
* __void schedule(TimerTask task, Date firstTime, long period):__ Schedule task for repeated fixed-delay execution, beginning at firstTime. Subsequent executions take place at approximately regular intervals, separated by period milliseconds. In fixed-delay execution, each execution is scheduled relative to the actual execution time of the previous execution. When an execution is delayed for any reason (such as garbage collection), subsequent executions are also delayed. In the long run, the frequency of execution will generally be slightly lower than the reciprocal of period (assuming the system clock underlying Object.wait(long) is accurate). As a consequence, when the scheduled firstTime value is in the past, task is scheduled for immediate execution. Fixed-delay execution is appropriate for recurring tasks that require “smoothness.” In other words, this form of execution is appropriate for tasks where it’s more important to keep the frequency accurate in the short run than in the long run. This includes most animation tasks, such as blinking a cursor at regular intervals. It also includes tasks wherein regular activity is performed in response to human input, such as automatically repeating a character for as long as a key is held down. This method throws IllegalArgumentException when firstTime.getTime() is negative or period is negative or zero; IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task-execution thread terminated; and NullPointerException when task or firstTime is null.
* __void schedule(TimerTask task, long delay):__ Schedule task for execution after delay milliseconds. This method throws IllegalArgumentException when delay is negative or delay + System.currentTimeMillis() is negative; IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task-execution thread terminated; and NullPointerException when task is null.
* __void schedule(TimerTask task, long delay, long period):__ Schedule task for repeated fixed-delay execution, beginning after delay milliseconds. Subsequent executions take place at approximately regular intervals separated by period milliseconds. This method throws IllegalArgumentException when delay is negative, delay + System.currentTimeMillis() is negative, or period is negative or zero; IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task-execution thread terminated; and NullPointerException when task is null.
* __void scheduleAtFixedRate(TimerTask task, Date firstTime, long period):__ Schedule task for repeated fixed- rate execution, beginning at time. Subsequent executions take place at approximately regular intervals, separated by period milliseconds. In fixed-rate execution, each execution is scheduled relative to the scheduled execution time of the initial execution. When an execution is delayed for any reason (such as garbage collection), two or more executions will occur in rapid succession to “catch up.” In the long run, the frequency of execution will be exactly the reciprocal of period (assuming the system clock underlying Object.wait(long) is accurate). As a consequence, when the scheduled firstTime is in the past, any “missed” executions will be scheduled for immediate “catch up” execution. Fixed-rate execution is appropriate for recurring activities that are sensitive to absolute time (such as ringing a chime every hour on the hour, or running scheduled maintenance every day at a particular time). It’s also appropriate for recurring activities where the total time to perform a fixed number of executions is important, such as a countdown timer that ticks once every second for 10 seconds. Finally, fixed-rate execution is appropriate for scheduling multiple repeating timer tasks that must remain synchronized with respect to one another. This method throws IllegalArgumentException when firstTime.getTime() is negative, or period is negative or zero; IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task-execution thread terminated; and NullPointerException when task or firstTime is null.
* __void scheduleAtFixedRate(TimerTask task, long delay, long period):__ Schedule task for repeated fixed-rate execution, beginning after delay milliseconds. Subsequent executions take place at approximately regular intervals, separated by period milliseconds. This method throws IllegalArgumentException when delay is negative, delay + System.currentTimeMillis() is negative, or period is negative or zero; IllegalStateException when task was already scheduled or canceled, the timer was canceled, or the task-execution thread terminated; and NullPointerException when task is null.

__■ Note timer tasks should complete quickly. When a timer task takes too long to complete, it “hogs” the timer’s task-execution thread, delaying the execution of subsequent timer tasks, which may “bunch up” and execute in rapid succession if and when the offending timer task finally completes.__

You can also call the following methods from within the overriding timer task’s run() method:
* __boolean cancel():__ Cancelthistimertask.Whenthetimertask has been scheduled for one-shot execution and hasn’t yet run or when it hasn’t yet been scheduled, it will never run.
* __long scheduledExecutionTime():__ Return the scheduled execution time of the most recent actual execution of this timer task. (When this method is invoked while timer task execution is in progress, the return value is the scheduled execution time of the ongoing timer task execution.) This method is typically invoked from within a task’s run() method to determine whether the current execution of the timer task is sufficiently timely to warrant performing the scheduled activity. For example, you would specify code similar to if (System.currentTimeMillis() -scheduledExecutionTime() >= MAX_TARDINESS) return;at the start of the run() method to abort the current timer task execution when it’s not timely. This method is typically not used in conjunction with fixed-delay execution repeating timer tasks because their scheduled execution times are allowed to drift over time and are thus not terribly significant. scheduledExecutionTime() returns the time at which the most recent execution of this timer task was scheduled to occur, in the format returned by java.util.Date.getTime(). The return value is undefined when the timer task has yet to commence its first execution.

#### Summary
The ThreadGroup class describes a thread group, which stores a set of threads. It simplifies thread management by applying method calls to all contained threads. You should avoid using thread groups because the most useful methods are deprecated and because of a race condition.

The ThreadLocal class describes a thread-local variable, which lets you associate per-thread data (such as a user ID) with a thread. It provides a separate storage slot to each thread that accesses the variable. Think of a thread-local variable as a multislot variable in which each thread can store a different value in the same variable. Each thread sees only its value and is unaware of other threads having their own values in this variable. Values stored in thread-local variables are not related. A parent thread can use the InheritableThreadLocal class to pass a value to a child thread.

It’s often necessary to schedule a task for one-shot execution or for repeated execution at regular intervals. Java 1.3 introduced the Timer Framework, which consists of Timer and TimerTask classes, to facilitate working with threads in a timer context.
Chapter 5 introduces the concurrency utilities and presents executors.